Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [37]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
from heapq import heappop, heappush
import numpy as np

In [38]:
PUZZLE_DIM = 4
RANDOMIZE_STEPS = 1000
action = namedtuple('Action', ['pos1', 'pos2'])

In [39]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_goal(state: np.ndarray) -> bool:
    """Check if the state matches the goal configuration."""
    goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    return np.array_equal(state, goal)

The Approach done is the A* search algorithm, where the heuristic function is the Manhattan distance

In [40]:
def manhattan_distance(state: np.ndarray) -> int:
    """Calculate the Manhattan distance of the current state to the goal state."""
    distance = 0
    goal_positions = {i: (i // PUZZLE_DIM, i % PUZZLE_DIM) for i in range(1, PUZZLE_DIM ** 2)}
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:
                goal_x, goal_y = goal_positions[value]
                distance += abs(x - goal_x) + abs(y - goal_y)
    return distance

def is_goal(state: np.ndarray) -> bool:
    """Check if the state matches the goal configuration."""
    goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    return np.array_equal(state, goal)

In [41]:
def solve_puzzle(initial_state: np.ndarray):
    """Solve the puzzle using A* search with a priority queue (heapq)."""
    open_list = []
    heappush(open_list, (manhattan_distance(initial_state), 0, initial_state.tobytes(), []))
    closed_set = set()
    closed_set.add(tuple(map(tuple, initial_state)))

    while open_list:
        heuristic, moves, current_state_bytes, path = heappop(open_list)
        current_state = np.frombuffer(current_state_bytes, dtype=int).reshape(initial_state.shape)

        if is_goal(current_state):
            return path, len(path), len(closed_set)

        for action in available_actions(current_state):
            next_state = do_action(current_state, action)
            next_path = path + [action]
            next_state_tuple = tuple(map(tuple, next_state))

            if next_state_tuple not in closed_set:
                heappush(open_list, (manhattan_distance(next_state) * 1.1 + len(next_path), len(next_path), next_state.tobytes(), next_path))
                closed_set.add(next_state_tuple)

    return None, -1, len(closed_set)

In [ ]:
# Initialize and randomize the starting state
initial_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    initial_state = do_action(initial_state, choice(available_actions(initial_state)))

print("Initial state:")
print(initial_state)
# Solve the puzzle
solution_path, quality, cost = solve_puzzle(initial_state)


print("Solution path:", solution_path)
print("Quality (number of moves):", quality)
print("Cost (total nodes evaluated):", cost)

Randomizing:   0%|          | 0/1000 [00:00<?, ?it/s]

Initial state:
[[ 3 12  6  4]
 [ 8 11 10  7]
 [15 13  2  5]
 [ 9 14  1  0]]
